# СОБИРАЊЕ НА ПОДАТОЦИ И КРЕИРАЊЕ НА ДАТАСЕТИ ОД ПРЕБАРАНИ САЈТОВИ КОИ ПРОДАВААТ ЛАПТОПИ ВО МАКЕДОНИЈА


In [1]:
!pip install selenium



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install pandas


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# КОРИСНИ ФУНКЦИИ И ИМПОРТИРАЊЕ НА БИБЛИОТЕКИ


**Импортирање на потребни библиотеки**

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import html
import re

**Функција за земање на капацитетот на меморија за секој лаптоп**

In [4]:
def extract_storage(title):
    if not isinstance(title, str):
        return None

    title = title.upper()
    title = title.replace(" / ", " ").replace("+", " ")
    title = title.replace("ТБ", "TB").replace("ГБ", "GB")  # за секој случај за случајна појава на кирлица
    title = re.sub(r'([A-Z])(\d+GB)', r'\1 \2', title)     #  за зборовите кои се споени или се без празно место меѓу нив.
    title = re.sub(r'(\d+GB)(\d+(?:GB|TB))', r'\1 \2', title)

    matches = re.findall(r'(\d+(?:\.\d+)?)(?:\s*)(TB|GB)(?:\s*(SSD|HDD)?)?', title)

    results = set()
    for size, unit, dtype in matches:
        try:
            num = float(size)
            if unit == "TB" or (unit == "GB" and num >= 128):
                disk_type = dtype.strip() if dtype else "SSD"
                results.add(f"{int(num) if num.is_integer() else num} {unit} {disk_type}")
        except:
            continue

    return " / ".join(sorted(results)) if results else None


**Функција за земање на РАМ меморија за секој лаптоп**

In [5]:

def extract_ram(title):
    if not isinstance(title, str):
        return None
    match = re.search(r'(?<!\d)(\d{1,2})\s?GB(?!\s?(SSD|HDD))', title, re.IGNORECASE)
    return match.group(1) + " GB" if match else None

**Функција за земање на брендот на секој лаптоп**

In [6]:
def extract_brand(title):
    if not isinstance(title, str):
        return "Notebook"
    title = title.lower()
    known_brands = ["hp", "lenovo", "asus", "acer", "apple", "dell", "msi", "gigabyte", "huawei", "microsoft", "razer"]
    if "thinkbook" in title or "loq" in title:
        return "Lenovo"
    for brand in known_brands:
        if brand in title:
            return brand.capitalize()
    match = re.search(r'лаптоп\s+([A-Za-z0-9]+)', title)
    if match:
        return match.group(1).capitalize()
    return title.split()[0].capitalize() if title else "Notebook"

**Функција за земање на процесорите на секој лаптоп**

In [7]:
def extract_cpu(title):
    if not isinstance(title, str):
        return "Unknown"

    title = title.upper().replace("®", "").replace("™", "").replace("-", " ")
    title = re.sub(r"\s+", " ", title)

    patterns = [
        r'INTEL CORE ULTRA \d+ \d{3,5}[A-Z]*',
        r'INTEL CORE I[3579] \d{3,5}[A-Z]*',
        r'INTEL CORE I[3579]',
        r'INTEL I[3579] \d{3,5}[A-Z]*',
        r'I[3579] \d{3,5}[A-Z]*',
        r'I[3579]-\d{3,5}[A-Z]*',
        r'INTEL \d',
        r'CORE I[3579] \d{3,5}[A-Z]*',
        r'CORE [3579]',
        r'INTEL [3579]',
        r'AMD RYZEN \d \d{3,5}[A-Z]*',
        r'RYZEN \d \d{3,5}[A-Z]*',
        r'RYZEN\d \d{3,5}[A-Z]*',
        r'RYZEN[3579] \d{3,5}[A-Z]*',
        r'RYZEN [3579]',
        r'RYZEN[3579]',
        r'R[3579] \d{3,5}[A-Z]*',
        r'R[3579]-\d{3,5}[A-Z]*',
        r'APPLE M[1234]',
        r'M[1234]',
        r'INTEL UHD',
        r'INTEL XE',
        r'VEGA \d+',
        r'ADRENO \d+',
        r'SNAPDRAGON\s+[A-Z]*\s?X?[A-Z]*[\w\-]+',
        r'CORE\s+ULTRA\s+[Uu]?\d[- ]?\d{3,5}[A-Z]*',
        r'ULTRA\s+\d\s+\d{3,5}[A-Z]*',
        r'U\d-\d{3,5}[A-Z]*',                 
        r'ULTRA\s+\d\s+\d{3,5}[A-Z]*',
        r'RYZEN\s+AI\s+\d\s+HX\s+\d{3}',      
        r'R\d-AI\s+\d{3}',                    
        r'R\d\s+AI\s+\d{3}',
        r'PENTIUM\s?\d{0,5}[A-Z]*',
        r'CELERON\s?\d{0,5}[A-Z]*'

    ]

    for pattern in patterns:
        match = re.search(pattern, title)
        if match:
            return match.group(0).strip()

    return "Unknown"

**Нормализација на колоната за цени**

In [8]:
def normalize_price(val):
    if not val or "No price" in val:
        return None
    return float(val.replace("ден.", "").replace("MKD", "").replace(",", "").replace(".", "").strip())

**СОБИРАЊЕ НА ПОДАТОЦИ И КРЕИРАЊЕ НА ДАТАСЕТ**

In [9]:
# ---- Setup ----
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
service = Service()
driver = webdriver.Chrome(service=service, options=options)
results = []

#------ВЕБ СКРЕЈПИНГ ЗА САЈТ:

# 1-------- ANHOCH --------
for page in range(1, 6):
    driver.get(f"https://www.anhoch.com/categories/site-laptopi/products?brand=&attribute=&toPrice=324980&inStockOnly=2&sort=latest&perPage=20&page={page}")
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    products = soup.find_all("div", class_="product-card")
    for product in products:
        title = product.get("title")
        price_tag = product.find("div", class_="product-price")
        price = price_tag.get_text(strip=True).split("ден")[0].strip() + " ден." if price_tag else "No price"
        discount_tag = price_tag.find("span", class_="previous-price") if price_tag else None
        discount_price = discount_tag.get_text(strip=True) if discount_tag else None
        p = normalize_price(price)
        dp = normalize_price(discount_price)
        if p and dp and dp > p:
            price, discount_price = discount_price, price
        results.append({
            "Prodavnica": "Anhoch",
            "Ime": title,
            "Brend": extract_brand(title),
            "Procesor": extract_cpu(title),
            "RAM_Memorija": extract_ram(title),
            "Kapacitet": extract_storage(title),
            "Cena": price,
            "Klub/Smart/Happy cena": discount_price
        })
        
# 2 -------- TEHNOMARKET --------
driver.get("https://www.tehnomarket.com.mk/category/4003/laptopi")
time.sleep(5)
soup = BeautifulSoup(driver.page_source, "html.parser")
products = soup.find_all("li", class_="span4")

for product in products:
    title_tag = product.find("div", class_="product-name")
    title = title_tag.text.strip() if title_tag else "No title"

    # Цени на лаптопите заедно со попуст цена ако ја има
    price_spans = product.find_all("span", class_="nm")
    if price_spans:
        price = price_spans[0].text.strip() + " ден." if len(price_spans) > 0 else "No price"
        discount_price = price_spans[1].text.strip() + " ден." if len(price_spans) > 1 else None
    else:
        price = "No price"
        discount_price = None

    # средување на цените ако има потреба
    p = normalize_price(price)
    dp = normalize_price(discount_price)
    if p and dp and dp > p:
        price, discount_price = discount_price, price

    results.append({
        "Prodavnica": "Tehnomarket",
        "Ime": title,
        "Brend": extract_brand(title),
        "Procesor": extract_cpu(title),
        "RAM_Memorija": extract_ram(title),
        "Kapacitet": extract_storage(title),
        "Cena": price,
        "Klub/Smart/Happy cena": discount_price
    })


# 3 -------- GJIRAFA50 --------
for page in range(1, 6):
    driver.get(f"https://gjirafa50.mk/gaming-laptop?pagenumber={page}")
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    products = soup.find_all("div", class_="item-box")
    for product in products:
        title_tag = product.find("h3", class_="product-title")
        title = title_tag.text.strip() if title_tag else "No title"
        price_tag = product.find("span", class_="price")
        price = price_tag.text.strip() if price_tag else "No price"
        discount_tag = product.find("span", class_="old-price")
        discount_price = discount_tag.text.strip() if discount_tag else None
        p = normalize_price(price)
        dp = normalize_price(discount_price)
        if p and dp and dp > p:
            price, discount_price = discount_price, price
        results.append({
            "Prodavnica": "Gjirafa50",
            "Ime": html.unescape(title),
            "Brend": extract_brand(title),
            "Procesor": extract_cpu(title),
            "RAM_Memorija": extract_ram(title),
            "Kapacitet": extract_storage(title),
            "Cena": price,
            "Klub/Smart/Happy cena": discount_price
        })

# 4 -------- NEPTUN --------
for page in range(1, 6):
    driver.get(f"https://www.neptun.mk/prenosni_kompjuteri.nspx?page={page}")
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    products = soup.find_all("div", class_="white-box")
    if not products:
        break

    for product in products:
        title_tag = product.find("h2", class_="product-list-item__content--title")
        title = title_tag.get_text(strip=True) if title_tag else "No title"

        # обична цена
        regular_price_tag = product.select_one(".newPriceModel .product-price__amount--value")
        price = regular_price_tag.get_text(strip=True) + " ден." if regular_price_tag else "No price"

        # Happy цена
        happy_price_tag = product.select_one(".HappyCard .product-price__amount--value")
        discount_price = happy_price_tag.get_text(strip=True) + " ден." if happy_price_tag else None

        # нормализирање и средување на приоритетот на цените 
        p = normalize_price(price)
        dp = normalize_price(discount_price)
        if p and dp and dp > p:
            price, discount_price = discount_price, price

        results.append({
            "Prodavnica": "Neptun",
            "Ime": title,
            "Brend": extract_brand(title),
            "Procesor": extract_cpu(title),
            "RAM_Memorija": extract_ram(title),
            "Kapacitet": extract_storage(title),
            "Cena": price,
            "Klub/Smart/Happy cena": discount_price
        })


# ---- ЕКСПОРТ ----
driver.quit()
df = pd.DataFrame(results)
df["Klub/Smart/Happy cena"] = df["Klub/Smart/Happy cena"].where(df["Klub/Smart/Happy cena"].notnull(), pd.NA)
df.to_csv("laptops_merged.csv", index=False, encoding='utf-8-sig')
print(df.head())
print(f"Vkupno vcitani podatoci: {len(df)}")
print("Zacuvano kako fajl : 'laptops_merged.csv'")


  Prodavnica                                                Ime   Brend  \
0     Anhoch  Notebook HP Pavilion 15 Ryzen7 7730U 16GB/512G...      Hp   
1     Anhoch  Notebook Lenovo IdeaPad Slim 3 Ryzen3 7320U/8G...  Lenovo   
2     Anhoch  Notebook Lenovo IdeaPad 1 Ryzen5 5500U 8GB/512...  Lenovo   
3     Anhoch  Notebook Lenovo IdeaPad Slim 3 Core 5 120U/16G...  Lenovo   
4     Anhoch  Notebook Lenovo LOQ Gaming i5-12450HX/16GB/512...  Lenovo   

       Procesor RAM_Memorija   Kapacitet            Cena Klub/Smart/Happy cena  
0  RYZEN7 7730U        16 GB  512 GB SSD  27.980,00 ден.        26.980,00 ден.  
1  RYZEN3 7320U         8 GB  256 GB SSD  16.980,00 ден.                  <NA>  
2  RYZEN5 5500U         8 GB  512 GB SSD  18.980,00 ден.        17.980,00 ден.  
3        CORE 5        16 GB  512 GB SSD  30.980,00 ден.                  <NA>  
4    I5 12450HX        16 GB  512 GB SSD  47.980,00 ден.        46.980,00 ден.  
Vkupno vcitani podatoci: 402
Zacuvano kako fajl : 'laptops_merg

# ПРЕДПРОЦЕСИРАЊЕ И СТАНДАРДИЗАЦИЈА НА ПОДАТОЦИ

In [10]:
# ВЧИТУВАЊЕ НА ПОДАТОЧНО МНОЖЕСТВО
df1 = pd.read_csv("laptops_merged.csv")

# ПРЕГЛЕД
print(df1.columns)
print(df1.head())


Index(['Prodavnica', 'Ime', 'Brend', 'Procesor', 'RAM_Memorija', 'Kapacitet',
       'Cena', 'Klub/Smart/Happy cena'],
      dtype='object')
  Prodavnica                                                Ime   Brend  \
0     Anhoch  Notebook HP Pavilion 15 Ryzen7 7730U 16GB/512G...      Hp   
1     Anhoch  Notebook Lenovo IdeaPad Slim 3 Ryzen3 7320U/8G...  Lenovo   
2     Anhoch  Notebook Lenovo IdeaPad 1 Ryzen5 5500U 8GB/512...  Lenovo   
3     Anhoch  Notebook Lenovo IdeaPad Slim 3 Core 5 120U/16G...  Lenovo   
4     Anhoch  Notebook Lenovo LOQ Gaming i5-12450HX/16GB/512...  Lenovo   

       Procesor RAM_Memorija   Kapacitet            Cena Klub/Smart/Happy cena  
0  RYZEN7 7730U        16 GB  512 GB SSD  27.980,00 ден.        26.980,00 ден.  
1  RYZEN3 7320U         8 GB  256 GB SSD  16.980,00 ден.                   NaN  
2  RYZEN5 5500U         8 GB  512 GB SSD  18.980,00 ден.        17.980,00 ден.  
3        CORE 5        16 GB  512 GB SSD  30.980,00 ден.                   NaN  
4  

In [11]:
df1

,Prodavnica,Ime,Brend,Procesor,RAM_Memorija,Kapacitet,Cena,Klub/Smart/Happy cena
0,Anhoch,Notebook HP Pavilion 15 Ryzen7 7730U 16GB/512G...,Hp,RYZEN7 7730U,16 GB,512 GB SSD,"27.980,00 ден.","26.980,00 ден."
1,Anhoch,Notebook Lenovo IdeaPad Slim 3 Ryzen3 7320U/8G...,Lenovo,RYZEN3 7320U,8 GB,256 GB SSD,"16.980,00 ден.",NaN
2,Anhoch,Notebook Lenovo IdeaPad 1 Ryzen5 5500U 8GB/512...,Lenovo,RYZEN5 5500U,8 GB,512 GB SSD,"18.980,00 ден.","17.980,00 ден."
3,Anhoch,Notebook Lenovo IdeaPad Slim 3 Core 5 120U/16G...,Lenovo,CORE 5,16 GB,512 GB SSD,"30.980,00 ден.",NaN
4,Anhoch,Notebook Lenovo LOQ Gaming i5-12450HX/16GB/512...,Lenovo,I5 12450HX,16 GB,512 GB SSD,"47.980,00 ден.","46.980,00 ден."
...,...,...,...,...,...,...,...,...
397,Neptun,No title,No,Unknown,NaN,NaN,No price,NaN
398,Neptun,No title,No,Unknown,NaN,NaN,No price,NaN
399,Neptun,No title,No,Unknown,NaN,NaN,No price,NaN
400,Neptun,No title,No,Unknown,NaN,NaN,No price,NaN


In [12]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Prodavnica             402 non-null    object
 1   Ime                    402 non-null    object
 2   Brend                  402 non-null    object
 3   Procesor               402 non-null    object
 4   RAM_Memorija           351 non-null    object
 5   Kapacitet              351 non-null    object
 6   Cena                   402 non-null    object
 7   Klub/Smart/Happy cena  220 non-null    object
dtypes: object(8)
memory usage: 25.3+ KB


In [13]:
df1.isnull().sum()

Prodavnica                 0
Ime                        0
Brend                      0
Procesor                   0
RAM_Memorija              51
Kapacitet                 51
Cena                       0
Klub/Smart/Happy cena    182
dtype: int64

**Чистење на цените**

In [14]:
def clean_price(value):
    if pd.isna(value):
        return None
    value = str(value).replace("ден.", "").replace("MKD", "").replace(",", "").replace(".", "").strip()
    try:
        return float(value)
    except ValueError:
        return None

df1["Cena"] = df1["Cena"].apply(clean_price)
df1["Klub/Smart/Happy cena"] = df1["Klub/Smart/Happy cena"].apply(clean_price)


In [15]:
df1

,Prodavnica,Ime,Brend,Procesor,RAM_Memorija,Kapacitet,Cena,Klub/Smart/Happy cena
0,Anhoch,Notebook HP Pavilion 15 Ryzen7 7730U 16GB/512G...,Hp,RYZEN7 7730U,16 GB,512 GB SSD,2798000.0,2698000.0
1,Anhoch,Notebook Lenovo IdeaPad Slim 3 Ryzen3 7320U/8G...,Lenovo,RYZEN3 7320U,8 GB,256 GB SSD,1698000.0,NaN
2,Anhoch,Notebook Lenovo IdeaPad 1 Ryzen5 5500U 8GB/512...,Lenovo,RYZEN5 5500U,8 GB,512 GB SSD,1898000.0,1798000.0
3,Anhoch,Notebook Lenovo IdeaPad Slim 3 Core 5 120U/16G...,Lenovo,CORE 5,16 GB,512 GB SSD,3098000.0,NaN
4,Anhoch,Notebook Lenovo LOQ Gaming i5-12450HX/16GB/512...,Lenovo,I5 12450HX,16 GB,512 GB SSD,4798000.0,4698000.0
...,...,...,...,...,...,...,...,...
397,Neptun,No title,No,Unknown,NaN,NaN,NaN,NaN
398,Neptun,No title,No,Unknown,NaN,NaN,NaN,NaN
399,Neptun,No title,No,Unknown,NaN,NaN,NaN,NaN
400,Neptun,No title,No,Unknown,NaN,NaN,NaN,NaN


**Чистење/Стандардизирање на колоната за РАМ меморија**

In [16]:
def standardize_ram(ram):
    if pd.isna(ram):
        return None
    match = re.search(r'(\d{1,2})', str(ram))
    return int(match.group(1)) if match else None

df1["RAM_Memorija"] = df1["RAM_Memorija"].apply(standardize_ram)


**Претворање на капацитетот во вкупно GB меморија**

In [17]:
def extract_total_storage_gb(val):
    if not isinstance(val, str):
        return None

    total = 0
    parts = val.upper().split("/")
    for part in parts:
        match = re.search(r'(\d+(?:\.\d+)?)\s*(TB|GB)', part)
        if match:
            size, unit = match.groups()
            gb = float(size) * (1024 if unit == "TB" else 1)
            total += gb
    return int(total) if total > 0 else None


df1["Kapacitet"] = df1["Kapacitet"].apply(extract_total_storage_gb)



**Стандардизирање на имињата на процесорите, почисто и порубстно**

In [18]:
def clean_procesor(cpu):
    if pd.isna(cpu) or cpu.strip().lower() == "unknown":
        return None
    cpu = cpu.strip().upper()
    
    # Стандардизирање на празни места
    cpu = re.sub(r'\s+', ' ', cpu)
    
    # Нормализирање на чести патерни
    cpu = cpu.replace("INTEL CORE ", "I").replace("CORE ", "I")
    cpu = cpu.replace("AMD RYZEN", "RYZEN")
    cpu = cpu.replace("RYZEN R", "RYZEN ")
    cpu = cpu.replace("M1", "M1").replace("M2", "M2").replace("M3", "M3")
    
    cpu = cpu.replace("HX ", "HX")

    return cpu.strip().title()

df1["Procesor"] = df1["Procesor"].apply(clean_procesor)


In [19]:
df1

,Prodavnica,Ime,Brend,Procesor,RAM_Memorija,Kapacitet,Cena,Klub/Smart/Happy cena
0,Anhoch,Notebook HP Pavilion 15 Ryzen7 7730U 16GB/512G...,Hp,Ryzen7 7730U,16.0,512.0,2798000.0,2698000.0
1,Anhoch,Notebook Lenovo IdeaPad Slim 3 Ryzen3 7320U/8G...,Lenovo,Ryzen3 7320U,8.0,256.0,1698000.0,NaN
2,Anhoch,Notebook Lenovo IdeaPad 1 Ryzen5 5500U 8GB/512...,Lenovo,Ryzen5 5500U,8.0,512.0,1898000.0,1798000.0
3,Anhoch,Notebook Lenovo IdeaPad Slim 3 Core 5 120U/16G...,Lenovo,I5,16.0,512.0,3098000.0,NaN
4,Anhoch,Notebook Lenovo LOQ Gaming i5-12450HX/16GB/512...,Lenovo,I5 12450Hx,16.0,512.0,4798000.0,4698000.0
...,...,...,...,...,...,...,...,...
397,Neptun,No title,No,None,NaN,NaN,NaN,NaN
398,Neptun,No title,No,None,NaN,NaN,NaN,NaN
399,Neptun,No title,No,None,NaN,NaN,NaN,NaN
400,Neptun,No title,No,None,NaN,NaN,NaN,NaN


**Чистење на имињата, односно тргање на префикси пред имињата на моделите**

In [20]:
def clean_name(name):
    if not isinstance(name, str):
        return name
    # Remove common clutter words from the beginning
    return re.sub(r'^(Notebook|Лаптоп|Laptop)\s+', '', name, flags=re.IGNORECASE).strip()

df1["Ime"] = df1["Ime"].apply(clean_name)


In [21]:
df1

,Prodavnica,Ime,Brend,Procesor,RAM_Memorija,Kapacitet,Cena,Klub/Smart/Happy cena
0,Anhoch,HP Pavilion 15 Ryzen7 7730U 16GB/512GB SSD/15....,Hp,Ryzen7 7730U,16.0,512.0,2798000.0,2698000.0
1,Anhoch,Lenovo IdeaPad Slim 3 Ryzen3 7320U/8GB/256GB S...,Lenovo,Ryzen3 7320U,8.0,256.0,1698000.0,NaN
2,Anhoch,Lenovo IdeaPad 1 Ryzen5 5500U 8GB/512GB SSD/15...,Lenovo,Ryzen5 5500U,8.0,512.0,1898000.0,1798000.0
3,Anhoch,Lenovo IdeaPad Slim 3 Core 5 120U/16GB/512GB S...,Lenovo,I5,16.0,512.0,3098000.0,NaN
4,Anhoch,Lenovo LOQ Gaming i5-12450HX/16GB/512GB/RTX405...,Lenovo,I5 12450Hx,16.0,512.0,4798000.0,4698000.0
...,...,...,...,...,...,...,...,...
397,Neptun,No title,No,None,NaN,NaN,NaN,NaN
398,Neptun,No title,No,None,NaN,NaN,NaN,NaN
399,Neptun,No title,No,None,NaN,NaN,NaN,NaN
400,Neptun,No title,No,None,NaN,NaN,NaN,NaN


In [22]:
df1.isnull().sum()

Prodavnica                 0
Ime                        0
Brend                      0
Procesor                  52
RAM_Memorija              51
Kapacitet                 51
Cena                      50
Klub/Smart/Happy cena    182
dtype: int64

In [23]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Prodavnica             402 non-null    object 
 1   Ime                    402 non-null    object 
 2   Brend                  402 non-null    object 
 3   Procesor               350 non-null    object 
 4   RAM_Memorija           351 non-null    float64
 5   Kapacitet              351 non-null    float64
 6   Cena                   352 non-null    float64
 7   Klub/Smart/Happy cena  220 non-null    float64
dtypes: float64(4), object(4)
memory usage: 25.3+ KB


In [24]:
df1 = df1[df1['Ime'] != "No title"]

In [25]:
df1.isnull().sum()

Prodavnica                 0
Ime                        0
Brend                      0
Procesor                   2
RAM_Memorija               1
Kapacitet                  1
Cena                       0
Klub/Smart/Happy cena    132
dtype: int64

**Податоците кои фалат за процесор, рам и капацитет се всушност податоци кај кои не беа наведени претходно наведените, додека во колоната за клуб цените, missing values се лаптопите кои не се на попуст и немаат клуб/haPPy цени**

# Експортирање

In [26]:
df1.to_csv("laptopi_test_sredeni.csv", index=False, encoding='utf-8-sig')